Создаем кластер в облаке 

После создания можно подключиться к головной ноде - `ssh sshuser@<hadoop-name>-ssh.azurehdinsight.net`

**ВАЖНО**: При следующем создании кластера нужно указать ту же самую запись хранения и тот же самый контейнер, чтобы все данные, с которомы вы работали в HDFS сохранились и вы могли продожить с ними работу.

Посмотреть на ваши данные в HDFS можно через BLOB-storage view в самом Azure.

Подгрузим данные с твитами в хадуп

```
sudo curl -O https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_{`seq -s , 1 13`}.csv
```

Подформатируем
```
for i in {1..13}; do tail -n +2 IRAhandle_tweets_$i.csv > tweets_$i.data; done
```

Отдельную папку для этих данных в HDFS
```
hdfs dfs -ls /
hdfs dfs -mkdir -p /tweets/data
```

Note: все команды для hdfs смотреть здесь - https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html

Заливаем данные
```
hdfs dfs -put tweets_* /tweets/data/
hdfs dfs -ls /tweets/data/
```

Скачиваем MR заадачи (например через гит или напрямую)
```
scp word-count.py sshuser@hadoop1-ssh.azurehdinsight.net:~/word-count.py
scp top-20.py  sshuser@hadoop1-ssh.azurehdinsight.net:~/
```

Собираем команду на запуск

```
yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=3 \
-files ~/word-count.py \
-mapper "python3 word-count.py map" \
-reducer "python3 word-count.py reduce" \
-input /tweets/data/ \
-output /tweets/result/
```

Смотрим результат
```
hdfs dfs -ls /tweets/result
hdfs dfs -cat /tweets/result/part-00000 | head
```

Запустим второй шаг
```
yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapreduce.job.name="top-20" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator="+" \
-files ~/top-20.py \
-mapper "python3 top-20.py map" \
-reducer "python3 top-20.py reduce" \
-input /tweets/result/ \
-output /tweets/top/
```

```
hdfs dfs -cat /tweets/top/part-00000
```

Чтобы посмотреть на консоль хадупа можно
* Добавить *.internal.cloudapp.net в прокси
* Добавить headnodehost в прокси
* Поднять прокси через головную ногу кластера
* Открыть `http://headnodehost:19888/jobhistory`
* Или открыть `http://hn1-hadoop.n3hsvtzmijuexf0fi4yqszkanb.bx.internal.cloudapp.net:8088/cluster` (ссылка может отличаться - она всегда пишется в консоли при запуске MR)

**Задача**. 

* Запустить MapReduce задачу с предыдущего семинара (биграммы).

In [ ]:
# DO IT (здесь хочется увидеть команды на запуск)

Далее будет работать с данным из комиссии по ценным бумагам и биржам США

Описанние данных лежит вот здесь - https://www.sec.gov/dera/data/edgar-log-file-data-set.html
Здесь порядка терабайла логов с сервера за месяц, разбитый под дням.

Важно заранее создать папку для данных

```
hdfs dfs -mkdir -p /seclog
```

Note: запускать необходимо с головной машины кластера
```
apt-get update && apt-get install parallel
```

```
printf %s\\n {01..30} | parallel -k -lb 'wget http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2017/Qtr2/log20170630.zip && unzip -p log201706{}.zip log201706{}.csv | tail -n +2 | hdfs dfs -put - /seclog/day_{}.csv && rm log201706{}.zip'
```

In [51]:
! hdfs dfs -cat /seclog/day_01.csv | head -n 15

104.197.198.jbj,2017-06-01,00:00:00,0.0,1622116.0,0001609253-17-000117,-index.htm,301.0,682.0,1.0,0.0,0.0,10.0,0.0,
104.197.198.jbj,2017-06-01,00:00:00,0.0,1622116.0,0001609253-17-000117,-index.htm,200.0,2878.0,1.0,0.0,0.0,10.0,0.0,
104.247.35.caa,2017-06-01,00:00:00,0.0,1071522.0,0001047469-98-043476,.txt,200.0,8444.0,0.0,0.0,0.0,10.0,0.0,
104.247.35.caa,2017-06-01,00:00:00,0.0,1071522.0,0001047469-98-043031,.txt,0.0,49756.0,0.0,0.0,0.0,10.0,0.0,
107.170.205.bch,2017-06-01,00:00:00,0.0,102037.0,0001140361-17-023189,.txt,301.0,673.0,0.0,0.0,0.0,10.0,0.0,
107.22.225.dea,2017-06-01,00:00:00,0.0,1576728.0,0001209191-17-036517,-index.htm,200.0,7702.0,1.0,0.0,0.0,10.0,0.0,
107.23.85.jfd,2017-06-01,00:00:00,0.0,1353678.0,0000914121-06-000548,-index.htm,200.0,2688.0,1.0,0.0,0.0,10.0,0.0,
107.23.85.jfd,2017-06-01,00:00:00,0.0,1059377.0,0001193125-15-111386,-index.htm,200.0,2741.0,1.0,0.0,0.0,10.0,0.0,
107.23.85.jfd,2017-06-01,00:00:00,0.0,1059377.0,0001193125-16-523709,-index.htm,200.0,2745.0,

Данные аккуратно прилетели. Описание столбцов можно найти вот здесь - https://www.sec.gov/files/EDGAR_variables_FINAL.pdf

**Важно (на семинаре)**: перед тем как начать решать следующую задачу, нужно поставить создаваться spark cluster - это долго.

**Задача**. 

* Посчитать на MapReduce средний объем данных (в байтах - смотри колонку size), который выкачивает каждый пользователь.


In [ ]:
# DO IT

После того, как кластер spark создался, можно начать пользоваться более удобный интерфейсом и работать в Jupyter, который хостится уже на кластере.

Он немножко сломан по умолчанию, но на официальном форуме рассказали, что это легко починить

Необходимо подключиться к головной машине через ssh, открыть файл `/usr/bin/anaconda/lib/python2.7/site-packages/nbformat/_version.py` и заменить 5 на 4.

После этого остается перезагрузить jupyter через ambari.

Сессия спарка доступна в ноутбуке через переменную `spark`.
Для того, чтобы спарк "прогрелся" и начал выполнять запросы, создадим контекст, который нам впоследствии потребуется.

In [ ]:
sp = spark.sparkContext

In [ ]:
data = sp.textFile("wasb:///seclog/day_01.csv")

Количество строк в файле

In [ ]:
data.count()

Предыдущая задача решенная на спарке:

In [ ]:
def get_user_and_size(line):
    columns = line.split(',')
    user, size = columns[0], columns[8]

    return user, float(size)

result = data.map(get_user_and_size).reduceByKey(lambda x, y: x+y).values().mean()
print(result)

Можно заметить, что получилось гораздо приятнее и быстрее, чем класический MR.

Полный список операций, которые можно делать на спарке - здесь 
* https://spark.apache.org/docs/latest/rdd-programming-guide.html
* http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD

Считаем общее количество пользователей

In [ ]:
total_users = data.map(lambda x: x.split(',')[0]).distinct().count()
print(total_users)

Считаем общее количество пользователей, которые сидят ночью

In [ ]:
total_night_users = data.filter(lambda x: int(x.split(',')[2].split(':')[0]) < 6).map(lambda x: x.split(',')[0]).distinct()\
                    .count()
print(total_night_users)

Считаем 10 самых больших документа, которые скачивали пользователи

In [ ]:
def get_name_and_size(line):
    columns = line.split(',')
    name, size = columns[6], columns[8]
    return float(size), name

top_10_large_result = data.map(get_name_and_size).sortByKey(ascending=False).values()\
                      .zipWithIndex().filter(lambda x: x[1] < 10).keys().collect()
print(result)

Помимо того, что результат можно получить в Jupyter, его можно положить в HDFS

In [56]:
! hdfs dfs -mkdir -p /seclogres

In [ ]:
result = data.map(get_name_and_size).sortByKey(ascending=False).values()\
        .zipWithIndex().filter(lambda x: x[1] < 10).keys()

result.saveAsTextFile("wasb:///seclogres/top_10_requests.txt")

In [64]:
! hdfs dfs -cat /seclogres/top_10_requests.txt/part-00000

.txt
.txt
.txt
.txt
.txt
.txt
.txt
.txt
.txt
.txt


**Задача**

* Посчитать топ 10 самых посещаемых страниц (считаются только успешные запросы - код 200)
* Посчитать количество людей по часам (~гистограма)
* Посчитать среднее количество людей по часам (~гистограма)

In [ ]:
# DO IT

Вычисления можно также проводить и в более "ручном" режиме (примерно как в MR)

Ниже - вычисление среднего объема, который выкачивает каждый пользователь (смотри задачу выше), решенная немного другим подходом

In [ ]:
def mapper(line):
    columns = line.split(',')
    user, size = columns[0], columns[8]
    return user, float(size)

def sum_reducer(item):
    key, values = item
    result = 0
    for value in values:
        result += value
    return result, 1

def mean_reducer(item):
    result_key, values = item
    summ, count = 0, 0
    for current_summ, current_count in values:
        summ += current_summ
        count += current_count
    return summ / count


result = data.map(mapper).groupByKey().map(sum_reducer).groupBy(lambda x: 1).map(mean_reducer).collect()
print(result)

**Задача**

* Скачать все логи за год в кластер. Найти всех пользователей, которые заходили каждый день месяца с указанием - что за месяц (если такие есть). Если таких нет - найти пользователей, которые заходили наибольшее в наибольшее количество дней (с указанием в какие месяца)

In [ ]:
# DO IT